### Setup

In [1]:
# this first setup block will pretty much always stay the same
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model_simplified import CovidModelSimplified
from covid_model.runnable_functions_simplified import do_single_fit, do_create_multiple_reports, do_fit_scenarios
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220921_scenario_projections.ipynb")

2022/09/08 09:31:06 INFO|============================================================


### Fit an initial scenario through Feb 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/input/params_simplified.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/input/vacc_proj_params_simplified.json',
    'start_date': '2020-01-24',
    'end_date': '2022-11-07',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-08-30'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/09/08 09:31:13 INFO|-------------------------|{} Retrieving vaccinations data
2022/09/08 09:31:16 INFO|-------------------------|{} Constructing vaccination projections
2022/09/08 09:31:16 INFO|-------------------------|{} Retrieving hospitalizations data
2022/09/08 09:31:18 INFO|----------------------|{} Prepping Model
2022/09/08 09:31:18 INFO|-----------------------|{} Prepping Model
2022/09/08 09:32:05 INFO|----------------------|{} Model prepped for fitting in 46.89446304100001 seconds.
2022/09/08 09:32:05 INFO|----------------------|{} Will fit 32 times
2022/09/08 09:34:30 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/32 completed in 144.357509792 seconds: {0: {'co': 4.957102226535142e-13}, 14: {'co': 0.7944347061020562}, 28: {'co': 2.3151295745434847e-11}, 42: {'co': 1.5843057077562832e-40}, 56: {'co': 0.3777831510333621}, 70: {'co': 0.6685375220569979}}
2022/09/08 09:34:30 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022

2022/09/08 09:38:53 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/32 completed in 14.35131287500002 seconds: {504: {'co': 0.6435429172756801}, 518: {'co': 0.6216310982096758}, 532: {'co': 0.608610430759221}, 546: {'co': 0.5441358124257989}, 560: {'co': 0.5708262509338289}, 574: {'co': 0.5704623502480274}}
2022/09/08 09:38:53 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/09/08 09:39:08 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/32 completed in 13.815453999999988 seconds: {532: {'co': 0.6082446840196855}, 546: {'co': 0.5451109124806198}, 560: {'co': 0.5689048420018281}, 574: {'co': 0.5766277375542206}, 588: {'co': 0.6158521276741261}, 602: {'co': 0.6055485778115539}}
2022/09/08 09:39:08 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/09/08 09:39:25 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/32 completed in 14.764668000000029 seconds: {

2022/09/08 09:42:19 INFO|-----------------------|{'fit_batch': '31', 'run_type': 'fit'}: Running forward sim
2022/09/08 09:42:21 INFO|----------------------|{'fit_batch': '31', 'run_type': 'fit'}: Uploading final results
2022/09/08 09:42:27 INFO|----------------------|{'fit_batch': '31', 'run_type': 'fit'}: spec_id: 3573


### Create and run scenarios from Feb 2022 to present

In [ ]:
multiprocess = 6

scenario_params = json.load(open("covid_model/analysis/20220815_aug_gov_briefing/20220815_scenario_params.json"))

model_args = {
    'base_spec_id': model.spec_id#3528 #model.spec_id # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# compute vacc effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for hosp_frac in [0.7, 0.8, 0.9]:
    for school_contact_increase in [1.0, 1.1, 1.2]:
        for beta_mult in [0.95, 1.0, 1.05]:
            hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*hosp_frac), "2022-03-15": (0.34 + 0.66*hosp_frac), "2022-03-30": hosp_frac}
            beta_param_adjustment = [{"param": "betta",
                                      "attrs": {"variant": "ba45"},
                                      "mults": {"2020-01-01": beta_mult},
                                      "desc": "sensitivity analysis for beta"}]
            school_contact = [{"param": "betta",
                               "attrs": {'age': '0-19'},
                               "mults": {"2020-01-01": 1, "2022-08-15": school_contact_increase},
                               "desc": "After August 15th, beta increases by a factor of the school contact increase"}]
            lt5_vacc_adjust = [{"param": "immunity",
                                "attrs": {'age': '0-19', 'vacc': 'shot1'},
                                "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                          "2022-06-30": 0.98 + 0.02*vacc_eff_lt5, "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                          "2022-07-20": 0.96 + 0.04*vacc_eff_lt5, "2022-07-30": 0.955 + 0.045*vacc_eff_lt5,
                                          "2022-08-10": 0.95 + 0.05*vacc_eff_lt5},
                                "desc": "Starting June 20th, the share of 0-19 getting vaccinated who are < 5 are weighted by the vaccine effectiveness multiplier for this age group"}]
            scenario_model_args.append({'params_defs': scenario_params + beta_param_adjustment + school_contact + lt5_vacc_adjust,
                                        'hosp_reporting_frac': hrf,
                                        'tags': {'beta_mult': beta_mult, 'hosp_frac': hosp_frac, 'school_contact_increase': school_contact_increase}})
            
            

In [ ]:
# check how many scenarios there are
len(scenario_model_args)

In [ ]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

### Run the report for each fit model

In [ ]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45'], from_date='2022-01-01')

In [ ]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-10-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-10-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')